In [1]:
import torch
import random
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("cerspense/zeroscope_v2_576w", torch_dtype=torch.bfloat16).to('cuda')
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
# pipe.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /share/u/rohit/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /share/u/rohit/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /share/u/rohit/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /share/u/rohit/.cache/huggingface/hub/models--cerspense--zeroscope_v2_576w/snapshots/6963642a64dbefa93663d1ecebb4ceda2d9ecb28/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
The TextToVideoSDPipeline has been deprecated and will not receive

# Original Model

In [2]:
prompt = "a scientist working in the lab"
seed = random.randint(0, 2**15)
video_frames = pipe(prompt, num_inference_steps=40, 
                    height=320, width=576, num_frames=24,
                    generator=torch.Generator().manual_seed(seed)).frames[0]
video_path = export_to_video(video_frames, "t2v_out_orig.mp4")

  0%|          | 0/40 [00:00<?, ?it/s]

It is recommended to use `export_to_video` with `imageio` and `imageio-ffmpeg` as a backend. 
These libraries are not present in your environment. Attempting to use legacy OpenCV backend to export video. 
Support for the OpenCV backend will be deprecated in a future Diffusers version


# Edited Model

In [9]:
from safetensors.torch import load_file
import copy

uce_path = '/share/u/rohit/uce_models/labcoats_uce_t2v.safetensors'
uce_weights = load_file(uce_path)
original_weights = {}
for name, param in pipe.unet.named_modules():
    if name+'.weight' in uce_weights:
        original_weights[name+'.weight'] = copy.deepcopy(param.weight)


pipe.unet.load_state_dict(uce_weights, strict=False)
print('Loaded UCE model weights')

Loaded UCE model weights


In [10]:
prompt = "a scientist working in the lab"
video_frames = pipe(prompt, num_inference_steps=40, 
                    height=320, width=576, num_frames=24,
                    generator=torch.Generator().manual_seed(seed)).frames[0]
video_path = export_to_video(video_frames, "t2v_out_edited_2.mp4")

  0%|          | 0/40 [00:00<?, ?it/s]

It is recommended to use `export_to_video` with `imageio` and `imageio-ffmpeg` as a backend. 
These libraries are not present in your environment. Attempting to use legacy OpenCV backend to export video. 
Support for the OpenCV backend will be deprecated in a future Diffusers version
